In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pylab as plt
import warnings
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
from matplotlib import pyplot as plt
warnings.simplefilter("ignore")
plt.style.use('ggplot')
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.width', 200)
pd.set_option('display.max_rows', None)

In [ ]:
train_transaction = pd.read_csv("../input/ieee-fraud-detection/train_transaction.csv")
test_transaction  = pd.read_csv("../input/ieee-fraud-detection/test_transaction.csv")

test_identity = pd.read_csv("../input/ieee-fraud-detection/test_identity.csv")
train_identity = pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")

sample_submission = pd.read_csv("../input/ieee-fraud-detection/sample_submission.csv")

In [ ]:
print('train_transaction shape is {}'.format(train_transaction.shape))
print('test_transaction shape is {}'.format(test_transaction.shape))
print('train_identity shape is {}'.format(train_identity.shape))
print('test_identity shape is {}'.format(test_identity.shape))

In [ ]:
# trainleri birleştirdik:
train = pd.merge(train_transaction,train_identity, on="TransactionID", how="left")
train.head()

In [ ]:
# testleri birleştirdik:
test = pd.merge(test_transaction,test_identity, on="TransactionID", how="left")
test.head()

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(train)

test = reduce_mem_usage(test)

In [ ]:
del train_identity, train_transaction, test_identity, test_transaction

############################ EDA ############################

In [ ]:
train.shape

In [ ]:
print(f'There are {train.isnull().any().sum()} columns in train dataset with missing values.')

In [ ]:
train.isnull().any().sum()/train.shape[1]

Kolonların %95'i eksik.

In [ ]:
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]
print(f'There are {len(one_value_cols)} columns in train dataset with one unique value.')
print(f'There are {len(one_value_cols_test)} columns in test dataset with one unique value.')

In [ ]:
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """


    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

In [ ]:
# eksiklik oranı
(train.isnull().sum() / train.shape[0] * 100).sort_values(ascending=False)

In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(train)

In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(test)

In [ ]:
train['isFraud'].value_counts(normalize=True)

veriseti dengesiz bir target.

In [ ]:
sns.countplot(x="isFraud", data=train).set_title('Distribution of Target')
plt.show()

In [ ]:
round((len(train[train['isFraud']==1])/len(train))*100,2)

In [ ]:
train.columns

In [ ]:
train['TransactionAmt'].isnull().any()

In [ ]:
train['DeviceType'].isnull().any()

In [ ]:
sns.countplot(train['DeviceType'])

In [ ]:
sns.countplot(test['DeviceType'])

In [ ]:
sns.boxplot(train['TransactionAmt'])
plt.show()

In [ ]:
def target_summary_with_num(dataframe, target, numerical_col):
    print(dataframe.groupby(target).agg({numerical_col: "mean"}), end="\n\n\n")

In [ ]:
target_summary_with_num(train, "isFraud", "TransactionAmt")

In [ ]:
### TransactionDT ###

plt.figure(figsize=(15,5))
sns.distplot(train["TransactionDT"])
sns.distplot(test["TransactionDT"])
plt.title('train vs test TransactionDT distribution')
plt.show()


86400: 2 tarih arasında zaman aralığıdır. Bu zaman aralığını saniye olarak varsayıyoruz. Doğrulamak için google trendi baz aldık. Zaman aralığını tarihe çevirdiğimizde fraud peak'i ve black friday peak'i benzer.

In [ ]:
# Notice the minimum value of TransactionDT is 86400 that happens to be the number of seconds of 1 day so we can assume that the unit of the column is second*
# TransactionDT data of train and test set are from different distribution as the graphs don't overlap.
# The data spreads across 396 days which is about 13 months
# Looking at the peaks at both ends, they're likely to be associated with a festive season, Black Friday or Christmas maybe

In [ ]:
#YEAR
train.groupby('DeviceInfo').agg({'TransactionDT':'min'}).sort_values('TransactionDT').head(20)
# A quick check shows that the Samsung Galaxy S8 ***(SAMSUNG SM-G892A Build/NRD90M)*** was released in the US the earliest on 21st April 2017 (Wikipedia). Since there's transactions made by this phone the 2nd day of the data's time, this data apparently can only be as old as ***April of 2017***
#SAMSUNG S8'in çıkış tarihi*

In [ ]:
#DATE
np.max(test['TransactionDT'])/86400 - np.min(train['TransactionDT'])/8640

In [ ]:
google trend fotosu gelecek. https://trends.google.com/trends/explore?cat=18&date=2017-04-01%202018-12-31&geo=US

In [ ]:
# gün bazında transaction dağılımı
# How about plotting the TransactionDT day wise?
plt.figure(figsize=(20,10))
train_1, test_1 = plt.hist(np.ceil(train['TransactionDT']/86400), bins=182), plt.hist(np.ceil(test['TransactionDT']/86400), bins=182)
plt.show()

In [ ]:
# **Assuming we're right that the peaks are associated with the Black Friday season. Let's see what are the dates we're looking at**
train_1[1][:182][train_1[0]> 6000]

In [ ]:
# Since 2017's Black Friday was 24th November that happens to be tally with the numbers above. Let us just assume that the data begins on November 1st of 2017, Let's have a look at the 2nd peak in the test data
test_peaks = test_1[1][:182][test_1[0]> 5000]
test_peaks

In [ ]:
[datetime.date(2017,11,1) + datetime.timedelta(days=x) for x in test_peaks.tolist()]
# [datetime.date(2018, 11, 23),
#  datetime.date(2018, 11, 26),
#  datetime.date(2018, 11, 27),
#  datetime.date(2018, 12, 1)]
# *23rd November 2018 is Black Friday, and 26th November 2018 is Cyber Monday, how cool is that**


 Conclusion: Based on the patterns found within the data, together with insights provided by Google Trends, the data's start date is likely 2017/11/1

In [ ]:
### PRODUCTCD ###
train['ProductCD'].isnull().any()

In [ ]:
## let's look at the unique values
train['ProductCD'].value_counts()

In [ ]:
# For now we don't know exactly what these values represent.

train.groupby('ProductCD') \
    ['TransactionID'].count() \
    .sort_index() \
    .plot(kind='barh',
          figsize=(15, 3),
         title='Count of Observations by ProductCD')
plt.show()

train.groupby('ProductCD')['isFraud'] \
    .mean() \
    .sort_index() \
    .plot(kind='barh',
          figsize=(15, 3),
         title='Percentage of Fraud by ProductCD')
plt.show()

*  W has the most number of observations, C the least.
*  ProductCD C has the most fraud with >11%
*  ProductCD W has the least with ~2%

In [ ]:
train.groupby('ProductCD')['isFraud'] \
    .value_counts() \
    .sort_values(ascending=False) \
    .plot(kind='bar',
          figsize=(15, 3),
         title='Percentage of Fraud-NoNFraud by ProductCD')
plt.show()

In [ ]:
# plot to see if any pattern in product type and fraud
sns.countplot(x='ProductCD',hue='isFraud',data=train)
plt.show()

# fraudlar genelde: W ve C 

In [ ]:
### CARD1-CARD6 ###

# We are told these are all categorical, even though some appear numeric.

card_cols = [c for c in train.columns if 'card' in c]
train[card_cols].head()

print(train.dtypes)


In [ ]:
color_idx = 0
for c in card_cols:
    if train[c].dtype in ['float16','int16']:
        train[c].plot(kind='hist',
                      title=c,
                      bins=50,
                      figsize=(15, 2),
                      color=color_pal[color_idx])
    color_idx += 1
    plt.show()
    
# Looking at the amount of unique values, I suppose card1 could be the Issuer Identification Number (IIN), encoded as a categorical variable with LabelEncoder.
#
# Furthermore, card3 or card5 could be the issue country of the card.
#
# We already know that card4 contains the card type (visa, mastercard, etc.), and card6 contains the card category (credit, debit, etc.).
#
# As there are a total of 6 card-variables, the question is what the other 2 variables are. Some options include:
# Individual account identifier
# Some kind of card identifier (e.g. Gold Card, Platinum Card, etc.)
# With some pivot analysis we should be able to verify some of these hypotheses. E.g. all cards of bank X should be issued in country Y.
#  'Card 1' column in Data Dictionary is given as Categorical but it is behaving like Continuous Data.
# Having '13553' unique Value

In [ ]:
train_transaction_fr = train.loc[train['isFraud'] == 1]
train_transaction_nofr = train.loc[train['isFraud'] == 0]
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 8))
train_transaction_fr.groupby('card4')['card4'].count().plot(kind='barh', ax=ax1, title='Count of card4 fraud')
train_transaction_nofr.groupby('card4')['card4'].count().plot(kind='barh', ax=ax2, title='Count of card4 non-fraud')
train_transaction_fr.groupby('card6')['card6'].count().plot(kind='barh', ax=ax3, title='Count of card6 fraud')
train_transaction_nofr.groupby('card6')['card6'].count().plot(kind='barh', ax=ax4, title='Count of card6 non-fraud')
plt.show()

In [ ]:
train.card6.value_counts()

In [ ]:
def target_summary_with_cat(dataframe, target, categorical_col):
    print(pd.DataFrame({"TARGET_MEAN": dataframe.groupby(categorical_col)[target].mean()}), end="\n\n\n")

In [ ]:
target_summary_with_cat(train, "isFraud", "card6")

en büyük fraud oranı: credit

In [ ]:
### addr1 & addr2 ###
# The data description states that these are categorical even though they look numeric. Could they be the address value?

In [ ]:
print(' addr1 - has {} NA values'.format(train['addr1'].isna().sum()))

In [ ]:
print(' addr2 - has {} NA values'.format(train['addr2'].isna().sum()))

In [ ]:
train['addr1'].plot(kind='hist', bins=500, figsize=(15, 2), title='addr1 distribution')
plt.show()

In [ ]:
train['addr2'].plot(kind='hist', bins=500, figsize=(15, 2), title='addr2 distribution')
plt.show()


In [ ]:
# Unique count of Regions:
train.addr1.nunique()

In [ ]:
# To find top 10 regions by count of transaction.
addr1_df = pd.DataFrame(train.addr1.value_counts())
addr1_df = addr1_df.rename_axis("region").reset_index()
addr1_df = addr1_df.sort_values(by=["addr1"], ascending=False).head(10)

In [ ]:
top_region_df = train[train["addr1"].isin(list(addr1_df["region"]))]

In [ ]:
def countplot_viz(
    data,
    xcolumn,
    xlabel,
    ylabel,
    title,
    hue=None,
    fontsize_label=16,
    fontsize_title=20,
    fontsize_text=12,
    rotation=45,
    figsize_x=12,
    figsize_y=5,
    palette="mako",
):
    """
    This function gets a Python Pandas dataframe and visualize a countplot.
    :param data: Dataframe to be analyze
    :param xcolumn: This column designates x axis column.
    :param xlabel: It designates name of x axis column.
    :param ylabel: It designates name of y axis column.
    :param title: This column designates name of graph.
    :param hue: Name of variables in `data` or vector data, optional Inputs for plotting long-form data.
    :param fontsize_label: It designates label size.
    :param fontsize_title: It designates title size.
    :param rotation: It designates rotation of graph.
    :param palette: It designates colors of graph.
    :return: This function doesn't return anything.
    """
    plt.figure(figsize=(figsize_x,figsize_y))
    
    g = sns.countplot(x=xcolumn, data=data, hue=hue, palette=palette)
    g.set_title(title, fontsize=19)
    g.set_xlabel(xlabel, fontsize=17)
    g.set_ylabel(ylabel, fontsize=17)
    g.set_xticklabels(g.get_xticklabels(), rotation=40, ha="right")
    plt.tight_layout()
    for p in g.patches:
        height = p.get_height()
        g.text(
            p.get_x() + p.get_width() / 2.0,
            height + 3,
            "{:1}".format(height),
            ha="center",
            fontsize=fontsize_text,
        )    
    if hue != None:
        g.legend(bbox_to_anchor=(1, 1), loc=1, borderaxespad=0)

In [ ]:
countplot_viz(
    top_region_df,
    "addr1",
    "Transaction Regions",
    "Freq",
    "Regions Distribution",
    palette="rocket",)

In [ ]:
# Unique count of Country:
train.addr2.nunique()

In [ ]:
# To find top 5 countries by count of transaction.
addr2_df = pd.DataFrame(train.addr2.value_counts())
addr2_df = addr2_df.rename_axis("country").reset_index()
addr2_df = addr2_df.sort_values(by=["addr2"], ascending=False).head()
top_country_df = train[
    train["addr2"].isin(list(addr2_df["country"]))]

In [ ]:
countplot_viz(
    top_country_df,
    "addr2",
    "Transaction Countries",
    "Freq",
    "Country Distribution",
    palette="rocket_r",)

In [ ]:
# Observation:
# addr1:
# There are 332 unique regions in dataframe. We show top 10 regions that have most transactions.
# addr2:
# There are 74 unique countries in dataframe. We show top 5 countries that have most transactions.

In [ ]:
### dist1 & dist2 ###

# tahmin: card sahibi ve işlem yapılan yer arasındaki uzaklık
# Plotting with logx to better show the distribution. Possibly this could be the distance of the transaction vs. the card owner's home/work address. This is just a guess.

train['dist1'].plot(kind='hist',
                                bins=5000,
                                figsize=(15, 2),
                                title='dist1 distribution',
                                color=color_pal[1],
                                logx=True)
plt.show()

In [ ]:
train['dist2'].plot(kind='hist',
                                bins=5000,
                                figsize=(15, 2),
                                title='dist2 distribution',
                                color=color_pal[1],
                                logx=True)
plt.show()


In [ ]:
### C1 - C14 ###
# Because we are provided many numerical columns, we can create a pairplot to plot feature interactions. I know these plots can be hard to read, but it is helpful for gaining intution about potential feature interactions and if certain features have more variance than others.
c_cols = [c for c in train if c[0] == 'C']
train[c_cols].head()

In [ ]:
# Sample 500 fraud and 500 non-fraud examples to plot
sampled_train = pd.concat([train.loc[train['isFraud'] == 0].sample(500),
          train.loc[train['isFraud'] == 1].sample(500)])
sns.pairplot(sampled_train,
             hue='isFraud',
            vars=c_cols)
plt.show()

In [ ]:
### D1-D9 ###  
print(d_cols)

In [ ]:
print(train[d_cols].dtypes)

In [ ]:
# Similarly for features D1-D9. In these plots we can see some linear and non-linear interactions between features. We may want to create additional features using these interactions if we think it would help our model better find relationship between fraud and non-fraud observations.

#d_cols = [c for c in train if c[0] == 'D']
train[d_cols].head()

sns.pairplot(sampled_train,
             hue='isFraud',
            vars=d_cols)
plt.show()

In [ ]:
d_cols = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15']

In [ ]:
### M1-M9 ###
# Values are T F or NaN
# Column M4 appears to be different with values like M2 and M0

m_cols = [c for c in train if c[0] == 'M']
train[m_cols].head()

(train[m_cols] == 'T').sum().plot(kind='bar',
                                              title='Count of T by M column',
                                              figsize=(15, 2),
                                              color=color_pal[3])
plt.show()


In [ ]:
(train[m_cols] == 'F').sum().plot(kind='bar',
                                              title='Count of F by M column',
                                              figsize=(15, 2),
                                              color=color_pal[4])
plt.show()


In [ ]:
(train[m_cols].isna()).sum().plot(kind='bar',
                                              title='Count of NaN by M column',
                                              figsize=(15, 2),
                                              color=color_pal[0])
plt.show()



In [ ]:
for i in m_cols:
    target_summary_with_cat(train, "isFraud", i)

aralarında anlamlı bir fark var. M1*

In [ ]:
# Looking at M4 column since it is different than the others
train.groupby('M4')['TransactionID'] \
    .count() \
    .plot(kind='bar',
          title='Count of values for M4',
          figsize=(15, 3))
plt.show()

In [ ]:
### V1 - V339 ###
# Lots of 1s 0s and Nans, some larger values
v_cols = [c for c in train if c[0] == 'V']
train[v_cols].head()
train['P_emaildomain'].value_counts()
sns.countplot(x='P_emaildomain',hue='isFraud',data=train)
plt.show()

In [ ]:
train[v_cols].describe()

In [ ]:
train['v_mean'] = train[v_cols].mean(axis=1)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(15, 6))
train.loc[train['isFraud'] == 1]['v_mean'] \
    .apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='log transformed mean of V columns - Fraud',
          ax=ax1)
train.loc[train['isFraud'] == 0]['v_mean'] \
    .apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='log transformed mean of V columns - Not Fraud',
          color=color_pal[5],
          ax=ax2)
plt.show()

In [ ]:
### Device type ###
train.groupby('DeviceType') \
    .mean()['isFraud'] \
    .sort_values() \
    .plot(kind='barh',
          figsize=(15, 5),
          title='Percentage of Fraud by Device Type')
plt.show()

In [ ]:
train["DeviceType"].value_counts()

In [ ]:
train.groupby('DeviceInfo') \
    .count()['TransactionID'] \
    .sort_values(ascending=False) \
    .head(20) \
    .plot(kind='barh', figsize=(15, 5), title='Top 20 Devices in Train')
plt.show()

In [ ]:
train['R_emaildomain'].value_counts()
sns.countplot(x='R_emaildomain',hue='isFraud',data=train)
plt.show()

In [ ]:
train['P_emaildomain'].value_counts()
sns.countplot(x='P_emaildomain',hue='isFraud',data=train)
plt.show()

# CORRELATİON

In [ ]:
# compile list of C columns
c_list = []
for i in range(1,15):
    c_list.append('C' + str(i))

In [ ]:
# compile list of D columns
d_list = []
for i in range(1,16):
    d_list.append('D' + str(i))

In [ ]:
# compile list of V columns
v_list = []
for i in range(1,340):
    v_list.append('V' + str(i))

In [ ]:
# compile list of id columns
id_list = []
for i in range(1,12):
    ii = str(i).zfill(2)
    id_list.append('id_' + str(ii))
id_list.extend(['id_13','id_14','id_17','id_18','id_19','id_20','id_21',\
                'id_22','id_24','id_25','id_26','id_32'])

In [ ]:
# create list of columns label for correlation check
colums_for_corr = ['isFraud','TransactionAmt','card1','card2','card3','card5','addr1','addr2','dist1','dist2']
colums_for_corr.extend(c_list)
colums_for_corr.extend(d_list)
colums_for_corr.extend(v_list)
colums_for_corr.extend(id_list)

In [ ]:
corr_matrix = train[colums_for_corr].corr()

In [ ]:
#Hedef özellik ile güçlü bir korelasyona sahip hiçbir özellik yok gibi görünüyor, bu yüzden onu devralmak için elimizden geldiğince çok özelliği tutmamız gerekiyor¶ 
#Daha fazla özelliği silmeyeceğiz 
#Kategorik özellikleri kodlamak için verileri tekrar birleştirme olacak

In [ ]:
# Let's plot a heatmap to have a sight about correlation. Generally speaking, C and D show a high internal correlation. The same for V, but with some exceptions.

sns.set(rc={'figure.figsize':(20,20)})
sns.heatmap(corr_matrix, cmap="RdBu")
plt.show()

In [ ]:
# From the global correlation matrix, let's extract just the isFraud column and see labels with correlation higher than 0.20. (also correlation < - 0.2 would be interesting, but the minimum value encountered was about -0.14.

isFraud_corr = corr_matrix['isFraud']
columns_with_correlation = isFraud_corr.loc[isFraud_corr >= 0.2]
print(columns_with_correlation.sort_values(ascending=False))

In [ ]:
# For categorical variables, let's explore via graph the correlation between labels and isFraud.
# First of all, let's create a function that counts the 1 and 0 values in isFraud for each label. Please note that this function neglect all labels that have at least one value with isFraud=0 but no one with isFraud=1. At this stage, this is not a high limitation as we are interested to check correlation with isFraud=1.

def check_corr_fraud(df, col):
    fr1 = (df.loc[df['isFraud'] == 1][col].value_counts())
    fr0 = (df.loc[df['isFraud'] == 0][col].value_counts())

    fr = pd.merge(fr1, fr0, how='left',
                  left_on=fr1.index,
                  right_on=fr0.index)

    fr = fr.rename(columns={'key_0': col,
                            f'{col}_x': 'Fraud',
                            f'{col}_y': 'Not Fraud'})

    fr['tot count'] = fr['Fraud'] + fr['Not Fraud']

    fr['% Fraud'] = 100 * np.divide(fr['Fraud'], fr['tot count'])

    fr['% Not Fraud'] = 100 * np.divide(fr['Not Fraud'], fr['tot count'])

    return fr


In [ ]:
# Let's prepare also a function ready to make plots for categorical variables. Note that the ylim property was set to [0,100], in order to avoid misleading charts. The large function is to make a larger graph (for many categorical variables).

def plot_corr_fraud(df, col):
    a = check_corr_fraud(train, col)

    sns.set(rc={'figure.figsize': (11.7, 8.27)})
    plt.bar(a[col], a['% Fraud'])
    plt.ylim([0, 100])
    plt.ylabel('% Fraud', fontsize=12)
    plt.xlabel(col, fontsize=12)
    plt.xticks(rotation=90)
    plt.subplots_adjust(bottom=0.4)

    return

In [ ]:
def plot_corr_fraud_large(df, col):
    a = check_corr_fraud(train, col)

    sns.set(rc={'figure.figsize': (101.7, 8.27)})
    plt.bar(a[col], a['% Fraud'])
    plt.ylim([0, 100])
    plt.ylabel('% Fraud', fontsize=12)
    plt.xlabel(col, fontsize=12)
    plt.xticks(rotation=90)
    plt.subplots_adjust(bottom=0.4)

    return

In [ ]:
# In the following, I will select just the graphs with relevant correlation between labels and isFraud=1.

plot_corr_fraud(train,'P_emaildomain')
plt.show()


In [ ]:
plot_corr_fraud(train,'R_emaildomain')
plt.show()


In [ ]:
# Some email domains look to be more exposed to fraudulent transactions.

plot_corr_fraud_large(train,'id_31')
plt.show()

In [ ]:
# Some browser look to be more exposed to fraudulent transactions.

plot_corr_fraud_large(train,'DeviceInfo')
plt.show()

In [ ]:
# Bazı cihazlar dolandırıcılık işlemlerine daha fazla maruz kalıyor gibi görünüyor.

# Conclusion
# From numerical variables, some from the V group showed an appreciable correlation with the isFraud label.
#From categorical variables, emaildomain and DeviceInfo showed an appreciable correlation with the isFraud label.

############################ FEATURE ENGINEERING ############################

In [ ]:
train['TransactionAmt_to_mean_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_mean_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('mean')
train['TransactionAmt_to_std_card1'] = train['TransactionAmt'] / train.groupby(['card1'])['TransactionAmt'].transform('std')
train['TransactionAmt_to_std_card4'] = train['TransactionAmt'] / train.groupby(['card4'])['TransactionAmt'].transform('std')

test['TransactionAmt_to_mean_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_mean_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('mean')
test['TransactionAmt_to_std_card1'] = test['TransactionAmt'] / test.groupby(['card1'])['TransactionAmt'].transform('std')
test['TransactionAmt_to_std_card4'] = test['TransactionAmt'] / test.groupby(['card4'])['TransactionAmt'].transform('std')

train['id_02_to_mean_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('mean')
train['id_02_to_mean_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('mean')
train['id_02_to_std_card1'] = train['id_02'] / train.groupby(['card1'])['id_02'].transform('std')
train['id_02_to_std_card4'] = train['id_02'] / train.groupby(['card4'])['id_02'].transform('std')

test['id_02_to_mean_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('mean')
test['id_02_to_mean_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('mean')
test['id_02_to_std_card1'] = test['id_02'] / test.groupby(['card1'])['id_02'].transform('std')
test['id_02_to_std_card4'] = test['id_02'] / test.groupby(['card4'])['id_02'].transform('std')

train['D15_to_mean_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('mean')
train['D15_to_mean_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('mean')
train['D15_to_std_card1'] = train['D15'] / train.groupby(['card1'])['D15'].transform('std')
train['D15_to_std_card4'] = train['D15'] / train.groupby(['card4'])['D15'].transform('std')

test['D15_to_mean_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('mean')
test['D15_to_mean_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('mean')
test['D15_to_std_card1'] = test['D15'] / test.groupby(['card1'])['D15'].transform('std')
test['D15_to_std_card4'] = test['D15'] / test.groupby(['card4'])['D15'].transform('std')

train['D15_to_mean_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('mean')
train['D15_to_mean_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('mean')
train['D15_to_std_addr1'] = train['D15'] / train.groupby(['addr1'])['D15'].transform('std')
train['D15_to_std_addr2'] = train['D15'] / train.groupby(['addr2'])['D15'].transform('std')

test['D15_to_mean_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('mean')
test['D15_to_mean_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('mean')
test['D15_to_std_addr1'] = test['D15'] / test.groupby(['addr1'])['D15'].transform('std')
test['D15_to_std_addr2'] = test['D15'] / test.groupby(['addr2'])['D15'].transform('std')
train[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = train['P_emaildomain'].str.split('.', expand=True)
train[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = train['R_emaildomain'].str.split('.', expand=True)
test[['P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3']] = test['P_emaildomain'].str.split('.', expand=True)
test[['R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']] = test['R_emaildomain'].str.split('.', expand=True)